### Installing packages


In [22]:
install.packages("tidyverse")
install.packages("cowplot")
install.packages("digest")
install.packages("infer")
install.packages("repr")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [23]:
library(cowplot)
library(digest)
library(infer)
library(repr)
library(tidyverse)

In [1]:
1 + 2

[1] 3

# Methods: plan

This report uses rigorous statistical inference methods such as hypothesis testing and confidence intervals to compare the mean of fixed acidity and volatile acidity for low-quality and high-quality red wine. Our data sources are reliable since they are sourced from the UC Irvine Machine Learning Repository. As a result, our reports are trustworthy. \


Although we Plot the relevant raw data and Compute estimates of the population mean. Our approach does not infer if competing hypotheses are reasonable and does not capture the uncertainty of the estimated parameters. We improve these weaknesses by doing hypothesis tests and confidence intervals. We chose to use bootstrap sampling for hypothesis testing and calculating confidence intervals.

We decided to make $\alpha = 0.1$ \
Two hypothesis tests are required since we look at fixed and volatile acidity. \
Hypothesis tests 1: \
$H_0 =$ high quality and low quality red wines have the same mean of fixed acidity\
$H_1 =$ high quality and low quality red wines have the different mean of fixed acidity 

Hypothesis tests 2: \
$H_0 =$ high quality and low quality red wines have the same mean of volatile acidity\
$H_1 =$ high quality and low quality red wines have the different mean of volatile acidity 


We decided to use $90 \%$ confidence interval. We will calculating the confidence interval of two sample mean for fixed acidity: mean of fixed acidity of low quality red wines and mean of fixed acidity of high quality red wine.
and the confidence interval of two sample mean for volatile acidity: mean of volatile acidity of low quality red wines and mean of volatile acidity of high quality red wine. Thus We can conduct that we are $90 \%$ “confident” that a $90 \% $confidence interval captures the value of the population parameter from what range.

- What do you expect to find? \
We expect the mean of fixed acidity and volatile acidity for low-quality and high-quality red wine are different.

- What impact could such findings have? \
Producers can improve their processes to achieve the fixed acidity and volatile acidity required for high quality of red wines.\
Consumers can decide the quality of red wine by determining whether or not the fixed acidity and volatile acidity meet the requirements for a high quality red wine.
- What future questions could this lead to?\
Exploring the relationship between acidity and sensation in red wines.